#IMPORTS

In [1]:
# run
!pip install av
!pip install visdom wandb

     |████████████████████████████████| 37.2 MB 60 kB/s 
     |████████████████████████████████| 676 kB 5.1 MB/s 
     |████████████████████████████████| 1.7 MB 52.7 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 139 kB 66.7 MB/s 
     |████████████████████████████████| 180 kB 70.0 MB/s 
     |████████████████████████████████| 97 kB 8.1 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=1e50f7994c426672ba6c497f7290c9f15c15d4ea95b88ab7a466390e5bda419f
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=7e1d0978d5b52e5661eeb6663fe578969c0b2b9fd6a7a2cca7da1f801b06168e
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel 

In [2]:
# run
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#LOADING DATA
`vos/all_frames/clips_all.zip` has all the `mp4` files from Youtube-VOS, while the frame zips have images for each video. The torchvision data loader (and CRW code) is setup to work with `mp4`. 

But `clips_all.zip` probably has train/valid/test clips. May want to filter only the train clips.

In [ ]:
%%capture
!mkdir vos_clips
!unzip /content/drive/Shareddrives/EECS\ 542/vos/all_frames/clips_all.zip -d vos_clips

In [ ]:
dataset = data.kinetics.Kinetics400("vos_clips", frames_per_clip=8, transform=None, extensions=('mp4'), frame_rate=8)

In [ ]:
torch.save(dataset, "drive/Shareddrives/EECS 542/vos_all_clips")

In [ ]:
!cd videowalk

/bin/bash: line 0: cd: videowalk: No such file or directory


In [ ]:
!python -W ignore "/content/drive/Shareddrives/EECS 542/videowalk-master/code/train.py" --data-path "vos_clips/" \
--frame-aug grid --dropout 0.1 --clip-len 4 --temp 0.05 \
--model-type scratch --workers 16 --batch-size 2  \
--cache-dataset --data-parallel  --lr 0.0001 \
--output-dir "/content/drive/Shareddrives/EECS 542/data-dump/"
#--visualize

Namespace(batch_size=2, cache_dataset=True, clip_len=4, clips_per_video=5, data_parallel=True, data_path='vos_clips/', device='cuda', dropout=0.1, epochs=25, fast_test=False, featdrop=0.0, flip=False, frame_aug='grid', frame_skip=8, frame_transforms='crop', head_depth=0, img_size=256, lr=0.0001, lr_gamma=0.3, lr_milestones=[20, 30, 40], lr_warmup_epochs=0, model_type='scratch', momentum=0.9, name='', optim='adam', output_dir='/content/drive/Shareddrives/EECS 542/data-dump/', partial_reload='', patch_size=[64, 64, 3], port=8095, print_freq=10, remove_layers=[], restrict=-1, resume='', server='localhost', sk_align=False, sk_targets=False, start_epoch=0, steps_per_epoch=10000000000.0, temp=0.05, visualize=False, weight_decay=0.0001, workers=16, zero_diagonal=False)
torch version:  1.9.0+cu111
torchvision version:  0.10.0+cu111
Preparing training dataloader
Frame transforms: [RandomResizedCrop(size=(256, 256), scale=(0.8, 0.95), ratio=(0.7, 1.3), interpolation=bilinear)] crop
Frame augs: C

In [ ]:
!nvidia-smi

Wed Oct 27 17:10:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Evaluation

In [11]:
%%capture
!mkdir davis
!unzip /content/drive/Shareddrives/EECS\ 542/DAVIS-2017-trainval-480p.zip

In [12]:
!mv DAVIS/* davis/

In [ ]:
!zip -r davis2017_trainval.zip /content/davis

In [22]:
!cp davis2017_trainval.zip /content/drive/Shareddrives/EECS\ 542/

In [5]:
!cp -r /content/drive/Shareddrives/EECS\ 542/data-dump .
!cp -r /content/drive/Shareddrives/EECS\ 542/videowalk-master/ .

In [13]:
#PRETRAINED_PATH = "data-dump/model_4_201.pth"
PRETRAINED_PATH = "videowalk-master/pretrained.pth"
TEST_PY = "videowalk-master/code/test.py"


DAVIS_FILE_PATH = "davis_vallist.txt"

In [18]:
!python $TEST_PY --filelist $DAVIS_FILE_PATH \
    --model-type scratch --resume $PRETRAINED_PATH --save-path /content/davis_results5 \
    --topk 10 --videoLen 20 --radius 12  --temperature 0.05 --cropSize 256

Using GPU 0
Context Length: 20 Image Size: 256
Arguments Namespace(batchSize=1, cropSize=256, device='cuda', filelist='davis_vallist.txt', finetune=0, gpu_id='0', head_depth=-1, imgSize=256, long_mem=[0], manualSeed=777, model_type='scratch', no_l2=False, norm_mask=False, pca_vis=False, radius=12.0, remove_layers=['layer4'], resume='videowalk-master/pretrained.pth', round=False, save_path='/content/davis_results5', temperature=0.05, texture=False, topk=10, videoLen=20, visdom=False, visdom_server='localhost', workers=4)
stride Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
stride Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
stride Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
stride Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
stride Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
stride Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 

In [19]:
# Convert
!python videowalk-master/code/eval/convert_davis.py \
     --in_folder davis_results5/ --out_folder davis_final5/ --dataset /content/davis

In [20]:
# Compute metrics
!python davis2017-evaluation/evaluation_method.py \
--task semi-supervised   --results_path davis_final5/ --set val \
--davis_path davis/

Using precomputed results...
--------------------------- Global results for val ---------------------------
 J&F-Mean  J-Mean  J-Recall  J-Decay  F-Mean  F-Recall  F-Decay
    0.474   0.478     0.544    0.208   0.469     0.527    0.241

---------- Per sequence results for val ----------
             Sequence  J-Mean  F-Mean
       bike-packing_1   0.391   0.584
       bike-packing_2   0.531   0.618
          blackswan_1   0.852   0.862
          bmx-trees_1   0.072   0.181
          bmx-trees_2   0.415   0.590
         breakdance_1   0.532   0.482
              camel_1   0.606   0.494
     car-roundabout_1   0.868   0.749
         car-shadow_1   0.784   0.738
               cows_1   0.809   0.671
        dance-twirl_1   0.276   0.336
                dog_1   0.825   0.751
          dogs-jump_1   0.162   0.219
          dogs-jump_2   0.425   0.357
          dogs-jump_3   0.572   0.619
      drift-chicane_1   0.603   0.586
     drift-straight_1   0.546   0.439
               goat_1   0.72

In [ ]:
# pretrained results
!python davis2017-evaluation/evaluation_method.py \
--task semi-supervised   --results_path davis_final2/ --set val \
--davis_path davis/

Using precomputed results...
--------------------------- Global results for val ---------------------------
 J&F-Mean  J-Mean  J-Recall  J-Decay  F-Mean  F-Recall  F-Decay
    0.172   0.188     0.184     0.24   0.156     0.109    0.235

---------- Per sequence results for val ----------
             Sequence  J-Mean  F-Mean
       bike-packing_1   0.000   0.000
       bike-packing_2   0.000   0.000
          blackswan_1   0.000   0.000
          bmx-trees_1   0.000   0.000
          bmx-trees_2   0.000   0.000
         breakdance_1   0.000   0.000
              camel_1   0.000   0.000
     car-roundabout_1   0.000   0.000
         car-shadow_1   0.000   0.000
               cows_1   0.000   0.000
        dance-twirl_1   0.047   0.048
                dog_1   0.622   0.388
          dogs-jump_1   0.066   0.100
          dogs-jump_2   0.099   0.096
          dogs-jump_3   0.539   0.601
      drift-chicane_1   0.019   0.032
     drift-straight_1   0.230   0.162
               goat_1   0.19

In [6]:
!cp -r /content/drive/Shareddrives/EECS\ 542/davis2017-evaluation .